In [1]:
# Tugas Besar IF3270 Pembelajaran Mesin
# Anggota Kelompok:
# 1. Gede Prasidha Bhawarnawa - 13520004

# Section Tugas Besar A: Implementasi Forward Propagation untuk Feed Forward Neural Network (FFNN)

# Import necessary libraries
import numpy as np

In [2]:
# Softmax Activation Function Implementation
def softmax(arr):
    numerator = np.exp(arr)
    denominator = np.sum(numerator)
    return numerator / denominator

In [3]:
# Sigmoid Activation Function Implementation
def sigmoid(arr):
    numerator = np.exp(arr)
    denominator = numerator + 1
    return numerator / denominator

In [4]:
# Linear Activation Function Implementation
def linear(arr):
    return arr

In [5]:
# Rectified Linear Unit (ReLU) Activation Function Implementation
def relu(arr):
    for i in range(len(arr)):
        if arr[i] < 0:
            arr[i] = 0
    return arr

In [6]:
# Format for the model input:

# number_of_input_nodes(INT)
# number_of_layers(INT)
# number_of_nodes_in_layer_1(INT)
# number_of_nodes_in_layer_2(INT)
# ...
# number_of_nodes_in_layer_n(INT)
# number_of_output_nodes(INT)
# INPUTS
# loop for nodes in input layer (
# input_node_number, input_node_value, input_node_weight_1, input_node_weight_2, ..., input_node_weight_n
# )
# HIDDEN
# loop for nodes in hidden layer (
# hidden_node_number, node_level, activation_function, node_weight_1, node_weight_2, ..., node_weight_n
# )

# Note that bias (both for input layer or hidden layer) are included in the input text file
# So, the number_of_nodes_in_layer_i is the number of nodes in the layer, including the bias node
# For each node, the number of weights is equal to the number of nodes in the next layer
# Hidden node number continues from the last node in the input layer
# All count start from 0 
# Input layer is layer 0, Output layer is layer (hidden_layer_count + 1)

# Example:

# 2
# 2
# 3
# 2
# 2
# INPUTS
# 1,10,0.1,0.2,0.3
# 2,-5,0.4,0.5,0.6
# HIDDEN
# 3,1,relu,0.7,0.8
# 4,1,relu,0.9,1.0
# 5,1,linear,1.1,1.2
# 6,2,linear,1.3,1.4
# 7,2,linear,1.5,1.6

In [19]:
# Node Class and Instances
class Node:
    def __init__(self, node_number:int, node_level, activation_function, weight):
        self.node_number = node_number
        self.node_level = node_level
        self.activation_function = activation_function
        self.weight = weight
        self.output = 0
        self.error = 0
    
    def getNodeNumber(self):
        return self.node_number
    
    def getNodeLevel(self):
        return self.node_level
    
    def getActivationFunction(self):
        functionList = {
            0: inputs,
            1: sigmoid,
            2: relu,
            3: linear,
            4: softmax,
            5: bias,
        }
        return functionList.get(self.activation_function, -1)
    
    def getWeight(self):
        return self.weight
    
    def getNodeInfo(self):
        print("Node Number: ", self.node_number)
        print("Node Level: ", self.node_level)
        print("Activation Function: ", self.activation_function)
        print("Weight: ", self.weight)
        print()

In [26]:
class Model:
    def __init__(self, number_of_layers:int, number_of_nodes:int, nodes:list):
        self.number_of_layers = number_of_layers
        self.number_of_nodes = number_of_nodes
        self.nodes = nodes # We assume that bias are also nodes of value 1
    
    def getNumberOfLayers(self):
        return self.number_of_layers
    
    def getNumberOfNodes(self):
        return self.number_of_nodes
    
    def getNodeList(self):
        return self.nodes
    
    def getNodeByIndex(self, index):
        return self.nodes[index]
    
    def getNodeByLevel(self, layer):
        nodes = []
        for node in self.nodes:
            if node.getNodeLevel() == layer:
                nodes.append(node)
        return nodes

    def getModelInfo(self):
        print("Number of Layers: ", self.number_of_layers)
        print("Number of Nodes: ", self.number_of_nodes) # Bias is also counted as a node here
        print("Nodes: ")
        for node in self.nodes:
            node.getNodeInfo()

In [24]:
# File reader and parser
from itertools import islice
import re
def readFile(filename):
    nodes =[]
    with open(filename, 'r') as f:
        contents = f.readlines()
        f.close()
    number_of_layers = int(contents[0])
    number_of_nodes = int(contents[1])
    for i in range(2, number_of_nodes + 2):
        node_info = contents[i].split(",")
        node = Node(int(node_info[0]), int(node_info[1]), node_info[2], float(node_info[3]))
        nodes.append(node)
    bias_weights = contents[number_of_nodes + 2].split(",")
    for layer in range(number_of_layers):
        number_of_nodes += 1
        node = Node(number_of_nodes, layer + 1, "bias", float(bias_weights[layer]))
        nodes.append(node)
    # for node in nodes:
    #     node.getNodeInfo()
    return Model(number_of_layers, number_of_nodes, nodes)
    

In [27]:
model = readFile('model1.txt')
model.getModelInfo()

Number of Layers:  2
Number of Nodes:  7
Nodes: 
Node Number:  1
Node Level:  1
Activation Function:  relu
Weight:  0.3

Node Number:  2
Node Level:  1
Activation Function:  sigmoid
Weight:  0.1

Node Number:  3
Node Level:  2
Activation Function:  linear
Weight:  0.5

Node Number:  4
Node Level:  2
Activation Function:  softmax
Weight:  0.7

Node Number:  5
Node Level:  2
Activation Function:  relu
Weight:  0.4

Node Number:  6
Node Level:  1
Activation Function:  bias
Weight:  0.2

Node Number:  7
Node Level:  2
Activation Function:  bias
Weight:  0.2

